# Worksheet 08

Name: Akhil Kokkula  
UID: U99436171

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [20]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.803676703030271, 4.621104692864513, 4.872316171122979, 4.01867377040279, 4.941984626325077, 6.255947746627925, 3.025498407623114, 6.8814942104720895, 5.480870844378939, 4.615243028107132]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [21]:
mean = 8
stdev = 1

s2 = np.random.normal(mean, stdev, 10).tolist()
print(s2)

[7.148151935767252, 6.9335066067686695, 7.722113041297566, 9.10291082279617, 5.957263428459406, 7.341128724050979, 6.286187070462093, 8.541044605880552, 6.845097322313139, 6.215527605515896]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [22]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.615243028107132, 5.480870844378939, 6.8814942104720895, 6.215527605515896, 6.845097322313139, 3.025498407623114, 6.255947746627925, 8.541044605880552, 6.286187070462093, 4.941984626325077]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.


- 1/2 chance from picking each normal distribution 
- One of the normal distributions has mean 5 and variance 1
- The other normal distribution has mean 8 and variance 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [23]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.8814942104720895, 6.215527605515896, 6.845097322313139, 6.255947746627925, 8.541044605880552, 6.286187070462093]
[4.615243028107132, 5.480870844378939, 3.025498407623114, 4.941984626325077]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 6.837549760211949,  mean_2 = 4.515899226608566
var_1 = 0.6555093414012906,  var_2 = 0.8359706957706603


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


They are not that close to the true values. P(S_j) should be 1/2 but it is 0.6 and 0.4. The mean should be 5 and 8, but it is 6.8 and 4.5. The variance should be 1 for both, but it is 0.66 and 0.84.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [24]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


#pretty off from the truth


point =  4.615243028107132
probability of observing that point if it came from cluster 0 =  0.0019433957210919033
probability of observing that point if it came from cluster 1 =  0.47386260300923544
point =  5.480870844378939
probability of observing that point if it came from cluster 0 =  0.07148025494541749
probability of observing that point if it came from cluster 1 =  0.2451228524600826
point =  6.8814942104720895
probability of observing that point if it came from cluster 0 =  0.6072328841745261
probability of observing that point if it came from cluster 1 =  0.008707742322933424
point =  6.215527605515896
probability of observing that point if it came from cluster 0 =  0.3879747826962855
probability of observing that point if it came from cluster 1 =  0.060412188357749814
point =  6.845097322313139
probability of observing that point if it came from cluster 0 =  0.6085585842377087
probability of observing that point if it came from cluster 1 =  0.009840121991263318
point =  3.02

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [33]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 6.7413909948889446,  mean_2 = 4.615642408784132
var_1 = 0.8192082724402762,  var_2 = 1.1395897756753288


They are not that close to the true values but slighlty improved. P(S_j) should be 1/2 but it is 0.6 and 0.4. The mean should be 5 and 8, but it is 6.7 and 4.6. The variance should be 1 for both, but it is 0.82 and 1.14.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [29]:
prob_s0_x = []
prob_s1_x = []

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

probabilities = zip(data, prob_s0_x, prob_s1_x)
for p in probabilities:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()

4.615243028107132
Probability of coming from S_0 = 0.02973974902703908
Probability of coming from S_1 = 0.9702602509729609

5.480870844378939
Probability of coming from S_0 = 0.40968770167630847
Probability of coming from S_1 = 0.5903122983236916

6.8814942104720895
Probability of coming from S_0 = 0.9687991864905303
Probability of coming from S_1 = 0.03120081350946968

6.215527605515896
Probability of coming from S_0 = 0.8606394447552903
Probability of coming from S_1 = 0.13936055524470967

6.845097322313139
Probability of coming from S_0 = 0.9663122275095932
Probability of coming from S_1 = 0.03368777249040684

3.025498407623114
Probability of coming from S_0 = 2.14799526375021e-05
Probability of coming from S_1 = 0.9999785200473624

6.255947746627925
Probability of coming from S_0 = 0.8730906610012963
Probability of coming from S_1 = 0.1269093389987038

8.541044605880552
Probability of coming from S_0 = 0.9973847278279016
Probability of coming from S_1 = 0.002615272172098452

6.2861

Not the best estimates but they are slightly better than the previous estimates.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [30]:
hard_assignments = []

for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0) 
    else:
        hard_assignments.append(1)

for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")

Data point 0 belongs to Cluster 1
Data point 1 belongs to Cluster 1
Data point 2 belongs to Cluster 0
Data point 3 belongs to Cluster 0
Data point 4 belongs to Cluster 0
Data point 5 belongs to Cluster 1
Data point 6 belongs to Cluster 0
Data point 7 belongs to Cluster 0
Data point 8 belongs to Cluster 0
Data point 9 belongs to Cluster 1
